# Трунов Никита
## Билет 0

## 1. Описание датасета и решаемой задачи 

Датасет включает в себя выборку автомобилей с различными харатеристиками. Решаемая задача - задача множественной регрессии. Целевая переменная - цена автомобиля.

## 2. Импорты

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import (
    make_scorer, r2_score, mean_absolute_error, mean_absolute_percentage_error,
    mean_squared_error)

## 3. Подготовительный этап

Загружаем датасет в Python.

In [2]:
df = pd.read_csv('CarPrice_Assignment.csv')
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


Исходный датасет содержит 205 точек данных и 26 переменных.

In [3]:
df.shape

(205, 26)

Отсутствующие значения в выборке не обнаружены.

In [4]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Исключаем ненужные переменные.

In [5]:
df = df.drop(columns=['car_ID'])

В столбце с названиями автомобилей достаточно много уникальных значений. Это плохой признак. Однако, можно выбрать из названий моделей марки автомобилей.

In [6]:
df['CarName'].nunique()

147

In [7]:
df['CarName'] = df['CarName'].str.split(' ',expand=True)[0].replace({
    'maxda': 'mazda',
    'nissan': 'Nissan',
    'porcshce': 'porsche',
    'toyouta': 'toyota', 
    'vokswagen': 'volkswagen',
    'vw': 'volkswagen'
})

In [8]:
df['CarName'].nunique()

22

Выделяем признаки и целевую переменную.

In [9]:
X = df.drop(columns=['price'])
y = df['price']

Используем способ "One Hot Encoding" для кодирования категориальных признаков.

In [10]:
X = pd.get_dummies(X)

Разделяем выборку на обучающую и тестовую в соотношении 80% и 20%.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

## 4. Обучение моделей

В качестве классификаторов будем использовать логистическую регрессию, метод опорных векторов с гауссовым ядром, k ближайших соседей и многослойных перцептрон.

In [12]:
rgs = (
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor()
)

В качестве метрик качества будем использовать коэффициент детерминации, корень из средней квадратичной ошибки RMSE (root mean squared error), cреднюю абсолютную ошибку MAE (mean absolute error) и среднюю абсолютную процентную ошибку MAPE (mean absolute percentage error).

In [13]:
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(mean_squared_error, squared=False),
    'mae': make_scorer(mean_absolute_error),
    'mape': make_scorer(mean_absolute_percentage_error)
}

Для определения эффективности моделей будем считать усредненные метрики в процессе проведения кросс-валидации. Она необходима, чтобы понимать, насколько полученные метрики валидны и подобрать на их основании наиболее эффективную модель.

In [14]:
scores = {}
for rg in rgs:
    metrics = {
        metric: values.mean()
        for metric, values in cross_validate(
            rg, X_train, y_train, scoring=scoring).items()
    }
    scores[type(rg).__name__] = metrics

Наилучшие показатели метрик оказались у RandomForestRegressor и GradientBoostingRegressor: коэффициент детерминации R2 наиболее близок к 1, а ошибки минимальны. Однако, время обучения RandomForestRegressor значительно больше, чем у GradientBoostingRegressor. Поэтому выбираем GradientBoostingRegressor.

In [15]:
pd.DataFrame(scores)

,LinearRegression,DecisionTreeRegressor,RandomForestRegressor,GradientBoostingRegressor
fit_time,0.007399,0.003601,0.198000,0.074194
score_time,0.003001,0.003200,0.012010,0.003000
test_r2,0.853163,0.842181,0.904592,0.904297
test_rmse,2965.985649,2965.521256,2276.259334,2263.999830
test_mae,2139.890707,1982.809316,1591.600617,1573.229577
test_mape,0.174069,0.139516,0.114120,0.115438


## 5. Построение итоговой модели

Обучим выбранную модель на тренировочных данных и сделаем предсказание на тестовых данных.

In [16]:
rg = GradientBoostingRegressor().fit(X_train, y_train)
y_pred = rg.predict(X_test)

Рассмотрим основные метрики эффективности: accuracy, precision, recall и f1.

In [17]:
r2_score(y_test, y_pred)

0.9142743501561925

In [18]:
mean_squared_error(y_test, y_pred, squared=False)

1616.337016801345

In [20]:
mean_absolute_error(y_test, y_pred)

1179.151268505089

In [21]:
mean_absolute_percentage_error(y_test, y_pred)

0.10007382888397491

Наблюдаются хорошие метрики, следовательно модель обучена хорошо. Обученная модель может быть применима в практических целях.